In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

In [ ]:
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'PUT HERE YOUR GOOGLE ID' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "data_csv.csv"):
      fileID = file['id']

data_csv = drive.CreateFile({'id':fileID})
data_csv.GetContentFile('data_news.csv')

In [18]:
import pandas as pd 
import sqlalchemy

server = 'LAPTOP-0CUGB1VO\SQLEXPRESS'
database = 'NSA'

data_csv = pd.read_csv('data_news.csv')
data_csv = data_csv.drop(['Unnamed: 0'], axis=1)

engine = sqlalchemy.create_engine('mssql+pyodbc://' + server + '/' + database + '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server')

data_csv.to_sql('NOTICIAS', con=engine, index=False, if_exists='append', chunksize=1000)

In [19]:
import pyodbc

conn = pyodbc.connect('Driver={SQL Server};'
                      f'Server={server};'
                      f'Database={database};'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

sql_query = pd.read_sql_query('SELECT * FROM NSA.dbo.NOTICIAS',conn)
sql_query

,ID,TITULO,RESUMO,SESSAO,DATA,PORTAL,LINK,SENTIMENT,PROBA_POSITIVA,PROBA_NEGATIVA
0,1,GLOBONEWS: assista às últimas notícias e análi...,None,GloboNews,2020-07-21 16:31:27.822554,G1,https://g1.globo.com/globonews/ao-vivo/globone...,1,0.582957,0.417043
1,2,Aneel proíbe corte de energia de famílias mais...,"Regra já estava em vigor, mas com prazo até o ...",Economia,2020-07-21 16:32:27.823123,G1,https://g1.globo.com/economia/noticia/2020/07/...,1,0.866448,0.133552
2,3,"Após decisão do Supremo, PGR busca dados da La...","No início do mês, Toffoli ordenou que todas as...",Operação lava jato,2020-07-21 15:31:27.823542,G1,https://g1.globo.com/politica/operacao-lava-ja...,1,0.819577,0.180423
3,4,"'Injeção de ânimo', diz 1ª voluntária a testar...",Médica de 27 anos está entre os 890 profission...,São Paulo,2020-07-21 15:31:27.824063,G1,https://g1.globo.com/sp/sao-paulo/noticia/2020...,0,0.024295,0.975705
4,5,Anvisa autoriza 3º teste clínico de vacina no ...,O que já se sabe sobre os avanços das vacinas ...,Coronavírus,2020-07-21 14:31:27.824678,G1,https://g1.globo.com/bemestar/coronavirus/noti...,1,0.598373,0.401627
...,...,...,...,...,...,...,...,...,...,...
542,543,"Corinthians x Palmeiras: onde assistir, histó...",Clássico marca o retorno do Campeonato Paulista,Esportes,21 de julho de 2020 07h30,Estadão,https://esportes.estadao.com.br/blogs/bate-pro...,1,0.598373,0.401627
543,544,Promotoria e PF miram caixa 2 de R$ 5 milhões...,"Operação Paralelo 23, terceira fase da operaçã...",Política,21 de julho de 2020 07h15,Estadão,https://politica.estadao.com.br/blogs/fausto-m...,1,0.591173,0.408827
544,545,'Desembargador carteirada': veja os memes e r...,Internautas ironizaram conduta de Eduardo Alme...,Política,21 de julho de 2020 07h15,Estadão,https://politica.estadao.com.br/blogs/fausto-m...,1,0.774218,0.225782
545,546,George Floyd: o cotidiano brasileiro,None,Política,21 de julho de 2020 07h15,Estadão,https://politica.estadao.com.br/blogs/fausto-m...,1,0.598373,0.401627
